# ResNet

## 一、为什么需要ResNet？

### 更深的网络不一定更好

```
正常的想法：
    10层网络 → 还行
    20层网络 → 更好
    50层网络 → 应该更更好？

实际情况：
    10层 → 还行
    20层 → 更好
    50层 → 反而变差了！

不是过拟合！是网络太深，梯度传不到底层，底层根本学不动
```

### ResNet怎么解决？

**加一条捷径，让输入可以跳过某些层直接传过去**

```
普通网络：
    x → 卷积 → 卷积 → 输出F(x)

ResNet：
    x → 卷积 → 卷积 → 结果F(x)
    x ─────────────────→ +       ← 把x加回来！
                          ↓
                    输出 = F(x) + x
```

### 为什么 +x 有用？

```
中间的层学到了东西：
    输出 = 有用的东西 + x → 比x更好 ✓

中间的层什么都没学到：
    输出 = 0 + x = x → 至少不会更差 ✓

所以加层只会变好或不变，永远不会变差
这就是为什么ResNet可以做到100层甚至1000层
```
---

## 二、残差连接长什么样？

```
输入x
  │
  ├─────────────────────────┐
  │                         │ 这条线就是"捷径"
  ↓                         │
Conv → BN → ReLU → Conv → BN   │
  │                         │
  ↓                         │
  + ←───────────────────────┘  把x加回来
  ↓
 ReLU
  ↓
 输出
```

**整个ResNet就是把很多这样的块堆在一起**

---

## 三、ResNet的整体结构

```
和LeNet套路一样：

LeNet：  Conv → Pool → Conv → Pool → Flatten → FC
ResNet： Conv → 残差块×很多 → 全局Pool → Flatten → FC

区别只是中间换成了残差块
```

### 数据怎么变的

```
通道越来越多：  64 → 128 → 256 → 512
空间越来越小：  56 → 28  → 14  → 7
最后：全局池化 → Flatten → Linear

跟LeNet一模一样的思路！
```

---

## 四、不同版本

```
ResNet18：18层，最快
ResNet34：34层，常用
ResNet50：50层，更强但更慢

数字 = 层数，越大越强越慢
实际中最常用：ResNet18 或 ResNet34
```

---



# 从最基础开始：搞懂训练到底在做什么


## 第一个问题：训练的完整流程是什么？

### 不管什么网络，训练永远是这4步

```
第1步：定义网络（告诉电脑你的模型长什么样）
第2步：准备数据（告诉电脑你的训练数据在哪）
第3步：训练（让电脑反复学习）
第4步：评估（看看学得怎么样）
```

**从LeNet到ResNet到任何网络，这4步永远不变！**

---

## 第二个问题：到底写不写网络？

### 两种方式，结果完全一样

**方式A：自己一层一层写（LeNet就是这么干的）**

```python
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2),
    nn.Sigmoid(),
    nn.AvgPool2d(2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5),
    nn.Sigmoid(),
    nn.AvgPool2d(2, stride=2),
    nn.Flatten(),
    nn.Linear(400, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10),
)
```
**方式B：直接用别人写好的结构（ResNet这么干）**

```python
net = models.resnet18()
net.fc = nn.Linear(512, 10)
```

```
PyTorch已经帮你把ResNet的结构写好了
你直接拿来用就行
就像买一套精装房，只需要换个门锁（改最后一层）
```

### 为什么ResNet不自己写？

```
LeNet：就5-6层，自己写很简单
ResNet18：18层，自己写很麻烦
ResNet50：50层，自己写要疯

所以PyTorch帮你写好了，直接调用就行
```

### 但是！两种方式后面的训练代码完全一样！

```python
# 不管你的net是怎么来的
# 后面这些代码一个字都不用变：

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(10):
    for X, y in train_loader:
        loss = loss_fn(net(X), y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
```

---

## 第三个问题：为什么要改最后一层？

### 用买房子来理解

```
PyTorch自带的ResNet18 = 一套精装房

这套房子原来是给"1000个人住的"（ImageNet有1000类）
你只需要"10个人住"（你的数据只有10类）

所以你要把最后一个房间改小一点
从1000人的改成10人的

net = models.resnet18()          # 买了一套1000人的房子
net.fc = nn.Linear(512, 10)      # 把最后一个房间改成10人的
```

### 改之前 vs 改之后

```
改之前：net的最后一层输出1000个数字（对应1000类）
改之后：net的最后一层输出10个数字（对应你的10类）

只改了最后一层！前面所有层都不用动！
```

---

## 第四个问题：net.train() 和 net.eval() 是什么？

```python
net.train()    # 告诉网络：现在是训练时间
net.eval()     # 告诉网络：现在是考试时间
```

### 为什么要区分？

```
训练时（net.train()）：
    BN层用当前batch的均值方差
    Dropout随机丢弃一些神经元
    → 正常学习

评估时（net.eval()）：
    BN层用全局的均值方差
    Dropout关闭（不丢弃）
    → 稳定地给出预测结果

如果评估时忘了写net.eval()：
    BN还在用batch的统计量 → 结果不稳定
    Dropout还在丢弃 → 结果变差
```

---

## 第五个问题：预训练和冻结是怎么回事？

### 现在不需要管！

```
这一节讲的ResNet = 从头训练
    net = models.resnet18()          ← 随机权重，从零开始学
    
后面"微调"那一节 = 加载预训练权重
    net = models.resnet18(pretrained=True)  ← 别人训练好的权重
    冻结前面的层
    只训练最后一层

这是两件完全不同的事！
现在只管"从头训练"就行！
```

---

## 把整个训练流程用做菜来比喻

```
第1步：选菜谱（定义网络）
    LeNet：简单家常菜的菜谱（自己写）
    ResNet：米其林大厨的菜谱（PyTorch写好的，直接用）

第2步：买菜（准备数据）
    train_loader = 买好的食材

第3步：做菜（训练）
    for epoch：做很多遍
        net(X)：按菜谱做一遍
        loss：尝一口，看看好不好吃
        backward：想想哪里要改
        step：调整一下

第4步：请人品尝（评估）
    net.eval()：告诉厨师现在是正式上菜
    看看准确率多少
```

**不管用什么菜谱（LeNet还是ResNet），做菜的步骤都一样！**

---

## 最终版代码模板（标注每一行在干什么）

```python
import torch
from torch import nn
import torchvision.models as models

# ====== 第1步：选菜谱 ======
# 方式A（简单网络，自己写）：
# net = nn.Sequential(nn.Conv2d(...), nn.ReLU(), ...)

# 方式B（复杂网络，用现成的）：
net = models.resnet18()              # 拿到ResNet的结构
net.fc = nn.Linear(512, 10)          # 改最后一层（10个类别）


# ====== 第2步：准备工具 ======
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(device)                       # 模型搬到GPU

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)  # 优化器
loss_fn = nn.CrossEntropyLoss()      # 损失函数


# ====== 第3步：训练 ======
for epoch in range(10):              # 学10遍
    net.train()                      # 切换到训练模式
    for X, y in train_loader:        # 每次拿一批数据
        X, y = X.to(device), y.to(device)   # 数据搬到GPU
        y_hat = net(X)               # 前向传播：预测
        loss = loss_fn(y_hat, y)     # 算损失：预测vs真实
        optimizer.zero_grad()        # 清零梯度
        loss.backward()              # 反向传播：算梯度
        optimizer.step()             # 更新权重


# ====== 第4步：评估 ======
    net.eval()                       # 切换到评估模式
    correct = 0
    total = 0
    with torch.no_grad():            # 评估时不需要算梯度
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            correct += (y_hat.argmax(dim=1) == y).sum().item()
            total += y.len()
    print(f'准确率: {correct/total}')
```

---

## 总结

```
1. 定义网络有两种方式：自己写 或 用现成的
   后面的训练代码完全一样

2. models.resnet18() = PyTorch帮你写好的ResNet结构
   不是预训练！就是个空壳子，权重是随机的

3. 改最后一层 = 把输出改成你的类别数
   因为默认是1000类，你可能只要10类

4. net.train() = 训练模式
   net.eval()  = 评估模式
   每次训练前写train()，评估前写eval()

5. 预训练、冻结、微调 = 后面的课才讲
   现在不用管！
```
---

## 七、什么时候改最后一层的512？

```python
net.fc = nn.Linear(512, 10)
#                  ^^^
#                  这个512哪来的？
```

```
ResNet18/34：最后一层输入是512   → nn.Linear(512, 类别数)
ResNet50/101/152：最后一层输入是2048 → nn.Linear(2048, 类别数)

不确定的话：
    print(net.fc)
    # 会告诉你 Linear(in_features=???, out_features=1000)
    # 那个???就是你要用的数字
```

---

## 总结

### 概念

```
ResNet = 每隔几层加一条捷径（输出 = F(x) + x）
好处：网络再深也能训练
是现代深度学习的标配
```

### 代码模板

```python
# 定义网络
net = models.resnet18()          # 或resnet34, resnet50
net.fc = nn.Linear(512, 类别数)   # 改最后一层

# 训练：和之前完全一样
net.to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
# for循环训练...
```

### 要记住的

```
1. ResNet的核心 = Y += X（残差连接）
2. 实际中直接用 models.resnet18()，不用自己写
3. 记得改最后一层 net.fc = nn.Linear(512, 你的类别数)
4. 训练代码和LeNet完全一样，一个字不用改
```